In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
avg_acc_list = [] 
avg_ang_acc_list = []
time_space_list = []
max_ang_acc_list = []

In [3]:
with open('./data/grade_and_comment.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
df = pd.DataFrame(data)

In [4]:
for i in range(0, len(data)):
    acc_x = np.array(data[i]["x軸平均加速度"]) 
    acc_y = np.array(data[i]["y軸平均加速度"])
    acc_z = np.array(data[i]["z軸平均加速度"])
    ang_acc_x = np.array(data[i]["x軸平均角加速度"])
    ang_acc_y = np.array(data[i]["y軸平均角加速度"]) 
    ang_acc_z = np.array(data[i]["z軸平均角加速度"])
  
    avg_acc = np.around((acc_x ** 2 + acc_y ** 2 + acc_z ** 2) ** 0.5, 3)
    avg_ang_acc = np.around((ang_acc_x ** 2 + ang_acc_y ** 2 + ang_acc_z ** 2) ** 0.5, 3)

    start_point = 0
    
    for num in data[i]["z軸平均加速度"]:
        if num < -3: #0
            break
        start_point += 1
    time_space = 51 - start_point
    avg_acc_list.append(avg_acc.tolist())
    avg_ang_acc_list.append(avg_ang_acc.tolist())
    time_space_list.append(time_space)
    max_ang_acc_list.append(avg_ang_acc[51])

In [5]:
add_data = {'平均加速度': avg_acc_list, '平均角加速度': avg_ang_acc_list, '起始點和擊球點時間間隔': time_space_list, '最大角加速度': max_ang_acc_list}
df_add = pd.DataFrame(add_data)
df = pd.concat([df,df_add],axis=1)

In [6]:
from tensorflow import keras
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Dropout
from keras.models import Sequential
from sklearn.model_selection import cross_val_score, KFold
from scikeras.wrappers import KerasRegressor
from keras.optimizers import Adam


In [7]:
def create_model(timestamp):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(timestamp, 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(50, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer= 'adam', loss='mse')
    return model

In [8]:
from sklearn.preprocessing import MinMaxScaler

In [9]:
scaler = MinMaxScaler()

x15_tmp = np.array(df[['平均加速度', '平均角加速度']].values.tolist()).reshape(-1, 250)
x15 = scaler.fit_transform(x15_tmp).reshape(-1, 250, 1) #normalization
x2_tmp = np.array(df['平均加速度'].tolist()).reshape(-1, 125)
x2 = scaler.fit_transform(x2_tmp).reshape(-1, 125, 1)
x3_tmp = np.array(df['平均角加速度'].tolist()).reshape(-1, 125)
x3 = scaler.fit_transform(x3_tmp).reshape(-1, 125, 1)
x4 = df[['起始點和擊球點時間間隔']]

y1 = df['揮拍軌跡正確度'].values.astype(int) / 5
y2 = df['揮拍速度流暢度'].values.astype(int) / 5
y3 = df['手腕轉動時機正確度'].values.astype(int) / 5 
y4 = df['擊球時機正確度'].values.astype(int) / 5
y5 = df['擊球位置正確度'].values.astype(int) / 5

In [10]:
from sklearn.linear_model import LinearRegression

In [11]:
# 將LSTM模型包裝在KerasRegressor中
regressor15 = KerasRegressor(build_fn = create_model(250), epochs=100, batch_size=16, verbose = 2) #100
regressor23 = KerasRegressor(build_fn = create_model(125), epochs=100, batch_size=16, verbose = 2) #100
regressor4 = LinearRegression()

kfold = KFold(n_splits = 10)



c:\Users\User\Desktop\badminton\badminton-venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
# cross validation
cv_result1 = cross_val_score(regressor15, x15, y1, cv = kfold, scoring='neg_root_mean_squared_error', verbose = 2, error_score='raise')
cv_result2 = cross_val_score(regressor23, x2, y2, cv = kfold, scoring='neg_root_mean_squared_error', verbose = 2, error_score='raise')
cv_result3 = cross_val_score(regressor23, x3, y3, cv = kfold, scoring='neg_root_mean_squared_error', verbose = 2, error_score='raise')
cv_result4 = cross_val_score(regressor4, x4, y4, cv = kfold, scoring='neg_root_mean_squared_error', verbose = 2, error_score='raise')
cv_result5 = cross_val_score(regressor15, x15, y5, cv = kfold, scoring='neg_root_mean_squared_error', verbose = 2, error_score='raise')

Epoch 1/100


c:\Users\User\Desktop\badminton\badminton-venv\lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 16 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
c:\Users\User\Desktop\badminton\badminton-venv\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


12/12 - 3s - 262ms/step - loss: 0.1097
Epoch 2/100
12/12 - 1s - 57ms/step - loss: 0.0737
Epoch 3/100
12/12 - 1s - 62ms/step - loss: 0.0442
Epoch 4/100
12/12 - 1s - 58ms/step - loss: 0.0591
Epoch 5/100
12/12 - 1s - 63ms/step - loss: 0.0576
Epoch 6/100
12/12 - 1s - 58ms/step - loss: 0.0494
Epoch 7/100
12/12 - 1s - 52ms/step - loss: 0.0463
Epoch 8/100
12/12 - 1s - 47ms/step - loss: 0.0473
Epoch 9/100
12/12 - 1s - 47ms/step - loss: 0.0493
Epoch 10/100
12/12 - 1s - 48ms/step - loss: 0.0494
Epoch 11/100
12/12 - 1s - 44ms/step - loss: 0.0581
Epoch 12/100
12/12 - 1s - 46ms/step - loss: 0.0570
Epoch 13/100
12/12 - 1s - 47ms/step - loss: 0.0520
Epoch 14/100
12/12 - 1s - 47ms/step - loss: 0.0480
Epoch 15/100
12/12 - 1s - 47ms/step - loss: 0.0483
Epoch 16/100
12/12 - 1s - 50ms/step - loss: 0.0466
Epoch 17/100
12/12 - 1s - 54ms/step - loss: 0.0476
Epoch 18/100
12/12 - 1s - 45ms/step - loss: 0.0471
Epoch 19/100
12/12 - 1s - 46ms/step - loss: 0.0472
Epoch 20/100
12/12 - 1s - 54ms/step - loss: 0.0469


c:\Users\User\Desktop\badminton\badminton-venv\lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 16 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
c:\Users\User\Desktop\badminton\badminton-venv\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


12/12 - 2s - 173ms/step - loss: 0.1107
Epoch 2/100
12/12 - 1s - 50ms/step - loss: 0.0703
Epoch 3/100
12/12 - 1s - 46ms/step - loss: 0.0555
Epoch 4/100
12/12 - 1s - 42ms/step - loss: 0.0513
Epoch 5/100
12/12 - 1s - 47ms/step - loss: 0.0588
Epoch 6/100
12/12 - 1s - 46ms/step - loss: 0.0460
Epoch 7/100
12/12 - 1s - 44ms/step - loss: 0.0512
Epoch 8/100
12/12 - 1s - 47ms/step - loss: 0.0434
Epoch 9/100
12/12 - 1s - 44ms/step - loss: 0.0475
Epoch 10/100
12/12 - 1s - 46ms/step - loss: 0.0520
Epoch 11/100
12/12 - 1s - 44ms/step - loss: 0.0543
Epoch 12/100
12/12 - 1s - 46ms/step - loss: 0.0453
Epoch 13/100
12/12 - 1s - 45ms/step - loss: 0.0546
Epoch 14/100
12/12 - 1s - 46ms/step - loss: 0.0494
Epoch 15/100
12/12 - 1s - 44ms/step - loss: 0.0548
Epoch 16/100
12/12 - 1s - 44ms/step - loss: 0.0711
Epoch 17/100
12/12 - 1s - 43ms/step - loss: 0.0723
Epoch 18/100
12/12 - 1s - 44ms/step - loss: 0.0641
Epoch 19/100
12/12 - 1s - 48ms/step - loss: 0.0586
Epoch 20/100
12/12 - 1s - 43ms/step - loss: 0.0531


...

12/12 - 2s - 154ms/step - loss: 0.0828
Epoch 2/100
12/12 - 1s - 44ms/step - loss: 0.0682
Epoch 3/100
12/12 - 1s - 42ms/step - loss: 0.0654
Epoch 4/100
12/12 - 1s - 43ms/step - loss: 0.0660
Epoch 5/100
12/12 - 0s - 42ms/step - loss: 0.0678
Epoch 6/100
12/12 - 1s - 46ms/step - loss: 0.0680
Epoch 7/100
12/12 - 1s - 46ms/step - loss: 0.0664
Epoch 8/100
12/12 - 0s - 42ms/step - loss: 0.0700
Epoch 9/100
12/12 - 0s - 41ms/step - loss: 0.0656
Epoch 10/100
12/12 - 1s - 43ms/step - loss: 0.0705
Epoch 11/100
12/12 - 1s - 58ms/step - loss: 0.0671
Epoch 12/100
12/12 - 1s - 42ms/step - loss: 0.0681
Epoch 13/100
12/12 - 1s - 42ms/step - loss: 0.0673
Epoch 14/100
12/12 - 1s - 44ms/step - loss: 0.0650
Epoch 15/100
12/12 - 0s - 42ms/step - loss: 0.0666
Epoch 16/100
12/12 - 1s - 42ms/step - loss: 0.0768
Epoch 17/100
12/12 - 1s - 42ms/step - loss: 0.0911
Epoch 18/100
12/12 - 1s - 44ms/step - loss: 0.0685
Epoch 19/100
12/12 - 1s - 42ms/step - loss: 0.0705
Epoch 20/100
12/12 - 0s - 41ms/step - loss: 0.0919


c:\Users\User\Desktop\badminton\badminton-venv\lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 16 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
c:\Users\User\Desktop\badminton\badminton-venv\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


12/12 - 2s - 145ms/step - loss: 0.0891
Epoch 2/100
12/12 - 1s - 47ms/step - loss: 0.0701
Epoch 3/100
12/12 - 1s - 44ms/step - loss: 0.0768
Epoch 4/100
12/12 - 0s - 42ms/step - loss: 0.0706
Epoch 5/100
12/12 - 0s - 41ms/step - loss: 0.0708
Epoch 6/100
12/12 - 1s - 44ms/step - loss: 0.0674
Epoch 7/100
12/12 - 1s - 42ms/step - loss: 0.0693
Epoch 8/100
12/12 - 0s - 42ms/step - loss: 0.0732
Epoch 9/100
12/12 - 0s - 40ms/step - loss: 0.0698
Epoch 10/100
12/12 - 1s - 43ms/step - loss: 0.0718
Epoch 11/100
12/12 - 1s - 46ms/step - loss: 0.0707
Epoch 12/100
12/12 - 1s - 44ms/step - loss: 0.0694
Epoch 13/100
12/12 - 1s - 42ms/step - loss: 0.0741
Epoch 14/100
12/12 - 1s - 46ms/step - loss: 0.0800
Epoch 15/100
12/12 - 1s - 43ms/step - loss: 0.0799
Epoch 16/100
12/12 - 1s - 42ms/step - loss: 0.0755
Epoch 17/100
12/12 - 0s - 39ms/step - loss: 0.0746
Epoch 18/100
12/12 - 1s - 45ms/step - loss: 0.0714
Epoch 19/100
12/12 - 1s - 44ms/step - loss: 0.0738
Epoch 20/100
12/12 - 0s - 39ms/step - loss: 0.0702


c:\Users\User\Desktop\badminton\badminton-venv\lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 16 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
c:\Users\User\Desktop\badminton\badminton-venv\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


12/12 - 2s - 148ms/step - loss: 0.0833
Epoch 2/100
12/12 - 1s - 43ms/step - loss: 0.0739
Epoch 3/100
12/12 - 1s - 47ms/step - loss: 0.0683
Epoch 4/100
12/12 - 1s - 44ms/step - loss: 0.0675
Epoch 5/100
12/12 - 1s - 44ms/step - loss: 0.0687
Epoch 6/100
12/12 - 1s - 42ms/step - loss: 0.0685
Epoch 7/100
12/12 - 0s - 41ms/step - loss: 0.0728
Epoch 8/100
12/12 - 1s - 45ms/step - loss: 0.0713
Epoch 9/100
12/12 - 1s - 44ms/step - loss: 0.0732
Epoch 10/100
12/12 - 1s - 43ms/step - loss: 0.0699
Epoch 11/100
12/12 - 0s - 40ms/step - loss: 0.0676
Epoch 12/100
12/12 - 1s - 44ms/step - loss: 0.0678
Epoch 13/100
12/12 - 0s - 41ms/step - loss: 0.0699
Epoch 14/100
12/12 - 1s - 42ms/step - loss: 0.0686
Epoch 15/100
12/12 - 0s - 41ms/step - loss: 0.0702
Epoch 16/100
12/12 - 1s - 46ms/step - loss: 0.0672
Epoch 17/100
12/12 - 1s - 42ms/step - loss: 0.0659
Epoch 18/100
12/12 - 1s - 42ms/step - loss: 0.0711
Epoch 19/100
12/12 - 0s - 39ms/step - loss: 0.0659
Epoch 20/100
12/12 - 1s - 43ms/step - loss: 0.0662


In [13]:
print("cv result1:", cv_result1)
print("cv result2:", cv_result2)
print("cv result3:", cv_result3)
print("cv result4:", cv_result4)
print("cv result5:", cv_result5)

cv result1: [-0.28635642 -0.16152801 -0.24248034 -0.30659419 -0.42895221 -0.17468632
 -0.32249031 -0.31304952 -0.29664794 -0.2       ]
cv result2: [-0.16630596 -0.18870338 -0.34928498 -0.14886055 -0.2144933  -0.20267728
 -0.16719995 -0.16030809 -0.18684806 -0.18037735]
cv result3: [-0.22266221 -0.24343646 -0.21612587 -0.23596245 -0.260468   -0.11060698
 -0.23320913 -0.23371763 -0.18265376 -0.16545753]
cv result4: [-0.20925183 -0.1807631  -0.23698441 -0.21760521 -0.2835228  -0.27698126
 -0.20169651 -0.30517981 -0.29330151 -0.23759789]
cv result5: [-0.21782019 -0.29301629 -0.2481718  -0.259344   -0.29073128 -0.20156334
 -0.2694952  -0.26745105 -0.16359018 -0.24772391]


In [14]:
regressor15.fit(x15[10:], y1[10:])
regressor23.fit(x2[10:], y2[10:])
regressor23.fit(x3[10:], y3[10:])
regressor4.fit(x4[10:], y4[10:])
regressor15.fit(x15[10:], y5[10:])

Epoch 1/100


c:\Users\User\Desktop\badminton\badminton-venv\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


12/12 - 3s - 263ms/step - loss: 0.1113
Epoch 2/100
12/12 - 1s - 42ms/step - loss: 0.0709
Epoch 3/100
12/12 - 1s - 47ms/step - loss: 0.0476
Epoch 4/100
12/12 - 1s - 47ms/step - loss: 0.0485
Epoch 5/100
12/12 - 1s - 44ms/step - loss: 0.0525
Epoch 6/100
12/12 - 1s - 44ms/step - loss: 0.0542
Epoch 7/100
12/12 - 1s - 44ms/step - loss: 0.0515
Epoch 8/100
12/12 - 1s - 42ms/step - loss: 0.0469
Epoch 9/100
12/12 - 1s - 43ms/step - loss: 0.0452
Epoch 10/100
12/12 - 1s - 43ms/step - loss: 0.0454
Epoch 11/100
12/12 - 1s - 43ms/step - loss: 0.0540
Epoch 12/100
12/12 - 1s - 42ms/step - loss: 0.0511
Epoch 13/100
12/12 - 1s - 43ms/step - loss: 0.0488
Epoch 14/100
12/12 - 1s - 45ms/step - loss: 0.0478
Epoch 15/100
12/12 - 1s - 42ms/step - loss: 0.0466
Epoch 16/100
12/12 - 0s - 41ms/step - loss: 0.0467
Epoch 17/100
12/12 - 1s - 45ms/step - loss: 0.0477
Epoch 18/100
12/12 - 1s - 58ms/step - loss: 0.0488
Epoch 19/100
12/12 - 1s - 56ms/step - loss: 0.0458
Epoch 20/100
12/12 - 1s - 49ms/step - loss: 0.0511


c:\Users\User\Desktop\badminton\badminton-venv\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


12/12 - 3s - 220ms/step - loss: 0.1038
Epoch 2/100
12/12 - 0s - 24ms/step - loss: 0.0852
Epoch 3/100
12/12 - 0s - 26ms/step - loss: 0.0524
Epoch 4/100
12/12 - 0s - 24ms/step - loss: 0.0505
Epoch 5/100
12/12 - 0s - 24ms/step - loss: 0.0499
Epoch 6/100
12/12 - 0s - 25ms/step - loss: 0.0471
Epoch 7/100
12/12 - 0s - 25ms/step - loss: 0.0430
Epoch 8/100
12/12 - 0s - 24ms/step - loss: 0.0429
Epoch 9/100
12/12 - 0s - 25ms/step - loss: 0.0426
Epoch 10/100
12/12 - 0s - 27ms/step - loss: 0.0387
Epoch 11/100
12/12 - 0s - 25ms/step - loss: 0.0418
Epoch 12/100
12/12 - 0s - 23ms/step - loss: 0.0438
Epoch 13/100
12/12 - 0s - 25ms/step - loss: 0.0444
Epoch 14/100
12/12 - 0s - 24ms/step - loss: 0.0392
Epoch 15/100
12/12 - 0s - 24ms/step - loss: 0.0406
Epoch 16/100
12/12 - 0s - 24ms/step - loss: 0.0413
Epoch 17/100
12/12 - 0s - 26ms/step - loss: 0.0385
Epoch 18/100
12/12 - 0s - 25ms/step - loss: 0.0460
Epoch 19/100
12/12 - 0s - 24ms/step - loss: 0.0363
Epoch 20/100
12/12 - 0s - 24ms/step - loss: 0.0383


c:\Users\User\Desktop\badminton\badminton-venv\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


12/12 - 0s - 23ms/step - loss: 0.0598
Epoch 2/100
12/12 - 0s - 27ms/step - loss: 0.0557
Epoch 3/100
12/12 - 0s - 23ms/step - loss: 0.0541
Epoch 4/100
12/12 - 0s - 23ms/step - loss: 0.0536
Epoch 5/100
12/12 - 0s - 24ms/step - loss: 0.0542
Epoch 6/100
12/12 - 0s - 23ms/step - loss: 0.0534
Epoch 7/100
12/12 - 0s - 24ms/step - loss: 0.0524
Epoch 8/100
12/12 - 0s - 24ms/step - loss: 0.0520
Epoch 9/100
12/12 - 0s - 26ms/step - loss: 0.0541
Epoch 10/100
12/12 - 0s - 22ms/step - loss: 0.0533
Epoch 11/100
12/12 - 0s - 25ms/step - loss: 0.0508
Epoch 12/100
12/12 - 0s - 24ms/step - loss: 0.0509
Epoch 13/100
12/12 - 0s - 23ms/step - loss: 0.0486
Epoch 14/100
12/12 - 0s - 24ms/step - loss: 0.0471
Epoch 15/100
12/12 - 0s - 24ms/step - loss: 0.0471
Epoch 16/100
12/12 - 0s - 26ms/step - loss: 0.0489
Epoch 17/100
12/12 - 0s - 24ms/step - loss: 0.0492
Epoch 18/100
12/12 - 0s - 24ms/step - loss: 0.0471
Epoch 19/100
12/12 - 0s - 24ms/step - loss: 0.0471
Epoch 20/100
12/12 - 0s - 24ms/step - loss: 0.0490
E

c:\Users\User\Desktop\badminton\badminton-venv\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


12/12 - 0s - 41ms/step - loss: 0.0519
Epoch 2/100
12/12 - 1s - 42ms/step - loss: 0.0596
Epoch 3/100
12/12 - 1s - 47ms/step - loss: 0.0533
Epoch 4/100
12/12 - 1s - 44ms/step - loss: 0.0535
Epoch 5/100
12/12 - 1s - 42ms/step - loss: 0.0749
Epoch 6/100
12/12 - 1s - 45ms/step - loss: 0.0572
Epoch 7/100
12/12 - 1s - 48ms/step - loss: 0.0556
Epoch 8/100
12/12 - 0s - 40ms/step - loss: 0.0550
Epoch 9/100
12/12 - 1s - 42ms/step - loss: 0.0537
Epoch 10/100
12/12 - 0s - 41ms/step - loss: 0.0548
Epoch 11/100
12/12 - 1s - 47ms/step - loss: 0.0494
Epoch 12/100
12/12 - 1s - 52ms/step - loss: 0.0571
Epoch 13/100
12/12 - 1s - 47ms/step - loss: 0.0534
Epoch 14/100
12/12 - 1s - 44ms/step - loss: 0.0516
Epoch 15/100
12/12 - 1s - 47ms/step - loss: 0.0534
Epoch 16/100
12/12 - 1s - 48ms/step - loss: 0.0531
Epoch 17/100
12/12 - 1s - 43ms/step - loss: 0.0517
Epoch 18/100
12/12 - 1s - 42ms/step - loss: 0.0475
Epoch 19/100
12/12 - 1s - 47ms/step - loss: 0.0503
Epoch 20/100
12/12 - 1s - 43ms/step - loss: 0.0545
E

KerasRegressor(
	model=None
	build_fn=<Sequential name=sequential, built=True>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=16
	validation_batch_size=None
	verbose=2
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=100
)

In [15]:
from joblib import dump, load

In [16]:
dump(regressor15, 'models/model15.joblib')
dump(regressor23, 'models/model23.joblib')
dump(regressor4, 'models/model4.joblib')

['models/model4.joblib']

In [17]:
loaded_regressor15 = load('models/model15.joblib')
loaded_regressor23 = load('models/model23.joblib')
loaded_regressor4 = load('models/model4.joblib')

In [18]:
ans1 = loaded_regressor15.predict(x15[:10])
print('ans1', ans1)
ans2 = loaded_regressor23.predict(x2[:10])
print('ans2', ans2)
ans3 = loaded_regressor23.predict(x3[:10])
print('ans3', ans3)
ans4 = loaded_regressor4.predict(x4[:10])
print('ans4', ans4)
ans5 = loaded_regressor15.predict(x15[:10])
print('ans5', ans5)

1/1 - 0s - 274ms/step
ans1 [0.50219995 0.48519486 0.67519546 0.43315172 0.9767207  0.88601875
 0.85406244 0.71588576 0.6858829  0.42291054]
1/1 - 0s - 388ms/step
ans2 [0.7195941  0.99657327 0.9976439  0.9533519  0.9314709  0.9967451
 0.9931126  0.98526996 0.91601926 0.78909713]
1/1 - 0s - 51ms/step
ans3 [0.5619056  0.60195386 0.66508853 0.5930299  0.93973017 0.9215253
 0.74011946 0.6501508  0.58747137 0.40442652]
ans4 [0.74110114 0.7549499  0.68570609 0.58184038 0.69955485 0.80342057
 0.69263047 0.68570609 0.69263047 0.56106724]
1/1 - 0s - 59ms/step
ans5 [0.50219995 0.48519486 0.67519546 0.43315172 0.9767207  0.88601875
 0.85406244 0.71588576 0.6858829  0.42291054]


In [19]:
answers = {'ans1': np.around(ans1 * 5, 1), 'ans2': np.around(ans2 * 5, 1), 'ans3': np.around(ans3 * 5, 1), 'ans4': np.around(ans4 * 5, 1), 'ans5': np.around(ans5 * 5, 1)}
df_answer = pd.DataFrame(answers)
print(df_answer)
print(round(df_answer.iloc[0]["ans1"], 1))

   ans1  ans2  ans3  ans4  ans5
0   2.5   3.6   2.8   3.7   2.5
1   2.4   5.0   3.0   3.8   2.4
2   3.4   5.0   3.3   3.4   3.4
3   2.2   4.8   3.0   2.9   2.2
4   4.9   4.7   4.7   3.5   4.9
5   4.4   5.0   4.6   4.0   4.4
6   4.3   5.0   3.7   3.5   4.3
7   3.6   4.9   3.3   3.4   3.6
8   3.4   4.6   2.9   3.5   3.4
9   2.1   3.9   2.0   2.8   2.1
2.5


In [20]:
#歐式距離(平方開根號)
from scipy.spatial import distance

In [21]:
#之後要把選擇的index記起來，如果要特定的index要用iloc or loc
min_d_acc_indexes = []
min_d_ang_indexes = []
min_space_indexes = []
acc_vectors = np.array(df['平均加速度'][10:].tolist())
ang_vectors = np.array(df['平均角加速度'][10:].tolist())

for target_vector in np.array(df['平均加速度'][:10].tolist()):
    distences = [distance.euclidean(target_vector, vec) for vec in acc_vectors]
    min_d_acc_indexes.append(np.argmin(distences))
for target_vector in np.array(df['平均角加速度'][:10].tolist()):
    distences = [distance.euclidean(target_vector, vec) for vec in ang_vectors]
    min_d_ang_indexes.append(np.argmin(distences))

for target in np.array(df['起始點和擊球點時間間隔'][:10]):
    min = float('inf')
    count = 0
    for value in np.array(df['起始點和擊球點時間間隔'][10:]):
        diff = abs(target - value)
        if diff < min:
           min = diff
           index = count
        count += 1
    min_space_indexes.append(index)

In [27]:
indexes = {'加速度索引': min_d_acc_indexes, '角加速度索引': min_d_ang_indexes, '起始點和擊球點時間間隔索引': min_space_indexes}
df_index = pd.DataFrame(indexes)
comments = []
ans_str = ('ans1', 'ans2', 'ans3', 'ans4', 'ans5')
score_str = ('揮拍軌跡', '揮拍速度', '手腕轉動時機', '擊球時機', '擊球位置')
for i in range(len(df_index)):
    comment = ''
    for string in str(df['評語'][df_index['加速度索引'][i]]).split('，'):
        if '軌跡' in string or '流暢' in string or '完整' in string:
            comment += string + ','
    for string in str(df['評語'][df_index['角加速度索引'][i]]).split('，'):
        if '手腕' in string or '軌跡' in string or '流暢' in string or '完整' in string:
            comment += string + ','
    for string in str(df['評語'][df_index['起始點和擊球點時間間隔索引'][i]]).split('，'):
        if '時間' in string or '時機' in string:
            comment += string + ','
    for i in range(5):
        if round(df_answer.iloc[i][ans_str[i]], 1) < 4:
            comment += score_str[i] + '有點不正確,'
        if round(df_answer.iloc[i][ans_str[i]], 1) < 3:
            comment += score_str[i] + '不正確,'
        if round(df_answer.iloc[i][ans_str[i]], 1) < 2:
            comment += score_str[i] + '完全不正確,'
    '''
    if round(df_answer.iloc[i]["ans1"], 1) < 4:
        comment += '揮拍軌跡不太正確' + ','
    if round(df_answer.iloc[i]["ans2"], 1) < 4:
        comment += '揮拍速度不太流暢' + ','
    if round(df_answer.iloc[i]["ans3"], 1) < 4:
        comment += '手腕轉動時機不太正確' + ','
    if round(df_answer.iloc[i]["ans4"], 1) < 4:
        comment += '擊球時機不太正確' + ','
    if round(df_answer.iloc[i]["ans5"], 1) < 4:
        comment += '擊球位置不太正確' + ','
    '''
    if len(comment) != 0:
        comment = comment[:-1]
    comments.append(comment)

TypeError: 'set' object is not subscriptable

In [ ]:
import google.generativeai as genai
import os

c:\Users\User\Desktop\badminton\badminton-venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
genai.configure(api_key=os.environ["GEMINI_API_KEY"])
model = genai.GenerativeModel('gemini-pro')
#print('origin comments:', comments)
count = 0
for comment in comments:
    if len(comment) == 0:
        res = model.generate_content(f'''現在你是一個羽球教練，要對學生的揮拍動作給出評語，如果這個學生高遠球的揮拍動作標準，那評語要給甚麼?
                                    給評語的注意事項:
                                    1.不要用條列式的方式 
                                    2.不要出現任何前情提要的字眼，直接講述評語就好 
                                    3.評語敘述不要太誇張，簡短且陳述事實就好''')
    else:
        res = model.generate_content(f'''現在你是一個羽球教練，要對學生的揮拍動作給出評語，雙引號內的文字內容是學生在高遠球揮拍時需改進的地方:"{comment}"。
                                    請根據雙引號內的內容，給出學生貼切的評語，讓他能知道高遠球揮拍動作能如何改進。
                                    給評語的注意事項:
                                    1.不要用條列式的方式 
                                    2.要出現任何前情提要的字眼，直接講述評語就好 
                                    3.盡量精簡''')
    print(f'''
          5個面向的分數(滿分5分):
          揮拍軌跡正確度:{round(df_answer.iloc[count]["ans1"], 1)}
          揮拍速度流暢度:{round(df_answer.iloc[count]["ans2"], 1)}
          手腕轉動時機正確度:{round(df_answer.iloc[count]["ans3"], 1)}
          擊球時機正確度:{round(df_answer.iloc[count]["ans4"], 1)}
          擊球位置正確度:{round(df_answer.iloc[count]["ans5"], 1)}
          評語:{res.text}''')
    count += 1


          5個面向的分數(滿分5分):
          揮拍軌跡正確度:3.3
          揮拍速度流暢度:3.2
          手腕轉動時機正確度:2.7
          擊球時機正確度:3.8
          擊球位置正確度:3.3
          評語:你的揮拍動作需要加強流暢性，並專注於維持正確的軌跡。留意手腕的轉動時機和擊球位置，確保在適當時機擊球，以達到最佳飛行距離。

          5個面向的分數(滿分5分):
          揮拍軌跡正確度:3.3
          揮拍速度流暢度:4.8
          手腕轉動時機正確度:4.5
          擊球時機正確度:3.9
          擊球位置正確度:3.3
          評語:你的高遠球揮拍還有進步空間。首先，你需要運用更多手腕力量，這將有助於你有效發力。此外，留意擊球時間點十分重要，它會影響球拍與球接觸的位置的準確性。同時，調整你的揮拍軌跡，確保能揮出一個自然的弧形，這樣才能穩定擊球並提高準度。最後，把握好擊球時機，這將幫助你產生理想的力道和距離。

          5個面向的分數(滿分5分):
          揮拍軌跡正確度:3.5
          揮拍速度流暢度:4.9
          手腕轉動時機正確度:4.0
          擊球時機正確度:3.4
          擊球位置正確度:3.5
          評語:你的高遠球揮拍還有進步的空間。揮拍軌跡不正確會影響擊球點和擊球力量。試著讓你的揮拍軌跡呈弧形，從低處將球挑起。同時，把握擊球時機，當球略低於頭部時揮拍擊球，這樣才能有足夠的時間揮動手臂，並將球擊出應有的高度。最後，調整你的擊球位置。高遠球應在後場底線附近擊球，站位太靠近網前會影響你的揮拍空間和擊球效果。

          5個面向的分數(滿分5分):
          揮拍軌跡正確度:3.2
          揮拍速度流暢度:4.7
          手腕轉動時機正確度:3.0
          擊球時機正確度:3.0
          擊球位置正確度:3.2
          評語:你的揮拍需要調整，以提升擊球質量。首先，確保手腕發力，讓拍面擊中球，並調整揮拍軌跡